### 训练后量化（PTQ）
准备模型：定义或加载预训练的模型。
配置量化：选择合适的量化配置，如量化数据类型、量化方案等。
插入量化节点：使用 torch.quantization.prepare 方法在模型中插入量化和反量化节点。
校准模型：使用校准数据集对模型进行校准，以确定量化参数（如缩放因子和零点）。
转换模型：使用 torch.quantization.convert 方法将模型转换为量化模型。

In [ ]:
import torch
import torch.nn as nn
import torch.quantization
from torchvision import datasets, transforms

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.conv = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.dequant(x)
        return x

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载数据集
train_dataset = datasets.MNIST(root='./data', train=True,
                               download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化模型
model = SimpleModel()

# 获取默认的量化配置
qconfig = torch.quantization.get_default_qconfig('x86')
model.qconfig = qconfig

# 准备模型
prepared_model = torch.quantization.prepare(model)  

# 校准模型（这里简单使用训练数据的一部分进行校准）
model.eval()
with torch.no_grad():
    for images, _ in train_loader:
        prepared_model(images)

# 转换为量化模型
quantized_model = torch.quantization.convert(prepared_model)